# Wide and Deep Model for Movie Recommendation

A linear model with a wide set of crossed-column (co-occurrence) features can memorize the feature interactions, while deep neural networks (DNN) can generalize the feature patterns through low-dimensional dense embeddings learned for the sparse features. [**Wide-and-deep**](https://arxiv.org/abs/1606.07792) learning jointly trains wide linear model and deep neural networks to combine the benefits of memorization and generalization for recommender systems.

Modified based on https://github.com/microsoft/recommenders/blob/master/examples/00_quick_start/wide_deep_movielens.ipynb

# Setup

In [101]:
import sys
import itertools
import math
import os
from tempfile import TemporaryDirectory
import numpy as np
import papermill as pm
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
DEFAULT_USER_COL = "userID"
DEFAULT_ITEM_COL = "itemID"
DEFAULT_RATING_COL = "rating"
DEFAULT_LABEL_COL = "label"
DEFAULT_TIMESTAMP_COL = "timestamp"
DEFAULT_PREDICTION_COL = "prediction"
USER_COL = "col_user"
ITEM_COL = "col_item"
RATING_COL = "col_rating"
PREDICT_COL = "col_prediction"
DEFAULT_K = 10
DEFAULT_THRESHOLD = 10
SEED = 42

# Data

In [17]:
ratings = pd.read_csv('data/ml-1m/ratings.dat', sep="::", header=None, engine='python')

In [19]:
ratings.columns = ["UserID", "MovieID", "Rating", "Timestamp"]

In [20]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   UserID     1000209 non-null  int64
 1   MovieID    1000209 non-null  int64
 2   Rating     1000209 non-null  int64
 3   Timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [23]:
movies = pd.read_csv('data/ml-1m/movies.dat', sep="::", header=None, engine='python')

In [26]:
movies.columns = ["MovieID", "Title", "Genres"]

In [27]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# Encode genres

In [28]:
# Encode 'genres' into int array (multi-hot representation) to use as item features
genres_encoder = sklearn.preprocessing.MultiLabelBinarizer()
movies["Genres"] = genres_encoder.fit_transform(movies["Genres"].apply(lambda s: s.split("|"))).tolist()
print("Genres:", genres_encoder.classes_)
display(movies.head())

Genres: ['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western']


,MovieID,Title,Genres
0,1,Toy Story (1995),"[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,Jumanji (1995),"[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,3,Grumpier Old Men (1995),"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,4,Waiting to Exhale (1995),"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,Father of the Bride Part II (1995),"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Merge data

In [39]:
data = pd.merge(ratings[["UserID", "MovieID", "Rating"]], movies[["MovieID", "Genres"]], on="MovieID")

In [40]:
data.head()

,UserID,MovieID,Rating,Genres
0,1,1193,5,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,1193,5,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,12,1193,4,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,15,1193,4,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,17,1193,5,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


# Train and test split

In [41]:
train, test = train_test_split(data, test_size=0.25)

In [42]:
items = data.drop_duplicates("MovieID")[["MovieID", "Genres"]].reset_index(drop=True)
item_feat_shape = len(items["Genres"][0])
users = data.drop_duplicates("UserID")[["UserID"]].reset_index(drop=True)
print("Total {} items and {} users in the dataset".format(len(items), len(users)))

Total 3706 items and 6040 users in the dataset


# Wide and deep model class

In [63]:
def build_feature_columns(
    users,
    items,
    user_col,
    item_col,
    item_feat_col=None,
    crossed_feat_dim=1000,
    user_dim=8,
    item_dim=8,
    item_feat_shape=None,
    model_type="wide_deep"
):
    """Build wide and/or deep feature columns for TensorFlow high-level API Estimator.

    Args:
        users (iterable): Distinct user ids.
        items (iterable): Distinct item ids.
        user_col (str): User column name.
        item_col (str): Item column name.
        item_feat_col (str): Item feature column name for 'deep' or 'wide_deep' model.
        crossed_feat_dim (int): Crossed feature dimension for 'wide' or 'wide_deep' model.
        user_dim (int): User embedding dimension for 'deep' or 'wide_deep' model.
        item_dim (int): Item embedding dimension for 'deep' or 'wide_deep' model.
        item_feat_shape (int or an iterable of integers): Item feature array shape for 'deep' or 'wide_deep' model.
        model_type (str): Model type, either
            'wide' for a linear model,
            'deep' for a deep neural networks, or
            'wide_deep' for a combination of linear model and neural networks.

    Returns:
        list of tf.feature_column, list of tf.feature_column: Two lists. One with the wide feature columns and a second
        with the deep feature columns. If only the wide model is selected, the deep column list is empty and viceversa. 
    """
    if model_type not in ["wide", "deep", "wide_deep"]:
        raise ValueError("Model type should be either 'wide', 'deep', or 'wide_deep'")

    user_ids = tf.feature_column.categorical_column_with_vocabulary_list(
        user_col, users
    )
    item_ids = tf.feature_column.categorical_column_with_vocabulary_list(
        item_col, items
    )

    if model_type == "wide":
        return _build_wide_columns(user_ids, item_ids, crossed_feat_dim), []
    elif model_type == "deep":
        return (
            [],
            _build_deep_columns(
                user_ids, item_ids, user_dim, item_dim, item_feat_col, item_feat_shape
            ),
        )
    elif model_type == "wide_deep":
        return (
            _build_wide_columns(user_ids, item_ids, crossed_feat_dim),
            _build_deep_columns(
                user_ids, item_ids, user_dim, item_dim, item_feat_col, item_feat_shape
            ),
        )


def _build_wide_columns(user_ids, item_ids, hash_bucket_size=1000):
    """Build wide feature (crossed) columns. `user_ids` * `item_ids` are hashed into `hash_bucket_size`

    Args:
        user_ids (tf.feature_column.categorical_column_with_vocabulary_list): User ids.
        item_ids (tf.feature_column.categorical_column_with_vocabulary_list): Item ids.
        hash_bucket_size (int): Hash bucket size.

    Returns:
        list of tf.feature_column: Wide feature columns.
    """
    # Including the original features in addition to the crossed one is recommended to address hash collision problem.
    return [
        user_ids,
        item_ids,
        tf.feature_column.crossed_column(
            [user_ids, item_ids], hash_bucket_size=hash_bucket_size
        ),
    ]


def _build_deep_columns(
    user_ids, item_ids, user_dim, item_dim, item_feat_col=None, item_feat_shape=1
):
    """Build deep feature columns

    Args:
        user_ids (tf.feature_column.categorical_column_with_vocabulary_list): User ids.
        item_ids (tf.feature_column.categorical_column_with_vocabulary_list): Item ids.
        user_dim (int): User embedding dimension.
        item_dim (int): Item embedding dimension.
        item_feat_col (str): Item feature column name.
        item_feat_shape (int or an iterable of integers): Item feature array shape.
    
    Returns:
        list of tf.feature_column: Deep feature columns.
    """
    deep_columns = [
        # User embedding
        tf.feature_column.embedding_column(
            categorical_column=user_ids, dimension=user_dim, max_norm=user_dim ** 0.5
        ),
        # Item embedding
        tf.feature_column.embedding_column(
            categorical_column=item_ids, dimension=item_dim, max_norm=item_dim ** 0.5
        ),
    ]
    # Item feature
    if item_feat_col is not None:
        deep_columns.append(
            tf.feature_column.numeric_column(
                item_feat_col, shape=item_feat_shape, dtype=tf.float32
            )
        )
    return deep_columns


def build_model(
    model_dir="model_checkpoints",
    wide_columns=(),
    deep_columns=(),
    linear_optimizer="Ftrl",
    dnn_optimizer="Adagrad",
    dnn_hidden_units=(128, 128),
    dnn_dropout=0.0,
    dnn_batch_norm=True,
    log_every_n_iter=1000,
    save_checkpoints_steps=10000,
    seed=None,
):
    """Build wide-deep model.
    
    To generate wide model, pass wide_columns only.
    To generate deep model, pass deep_columns only.
    To generate wide_deep model, pass both wide_columns and deep_columns.

    Args:
        model_dir (str): Model checkpoint directory.
        wide_columns (list of tf.feature_column): Wide model feature columns.
        deep_columns (list of tf.feature_column): Deep model feature columns.
        linear_optimizer (str or tf.train.Optimizer): Wide model optimizer name or object.
        dnn_optimizer (str or tf.train.Optimizer): Deep model optimizer name or object.
        dnn_hidden_units (list of int): Deep model hidden units. E.g., [10, 10, 10] is three layers of 10 nodes each.
        dnn_dropout (float): Deep model's dropout rate.
        dnn_batch_norm (bool): Deep model's batch normalization flag.
        log_every_n_iter (int): Log the training loss for every n steps.
        save_checkpoints_steps (int): Model checkpoint frequency.
        seed (int): Random seed.

    Returns:
        tf.estimator.Estimator: Model
    """
    # TensorFlow training log frequency setup
    config = tf.estimator.RunConfig(
        tf_random_seed=seed,
        log_step_count_steps=log_every_n_iter,
        save_checkpoints_steps=save_checkpoints_steps,
    )

    if len(wide_columns) > 0 and len(deep_columns) == 0:
        model = tf.estimator.LinearRegressor(
            model_dir=model_dir,
            config=config,
            feature_columns=wide_columns,
            optimizer=linear_optimizer,
        )
    elif len(wide_columns) == 0 and len(deep_columns) > 0:
        model = tf.estimator.DNNRegressor(
            model_dir=model_dir,
            config=config,
            feature_columns=deep_columns,
            hidden_units=dnn_hidden_units,
            optimizer=dnn_optimizer,
            dropout=dnn_dropout,
            batch_norm=dnn_batch_norm,
        )
    elif len(wide_columns) > 0 and len(deep_columns) > 0:
        model = tf.estimator.DNNLinearCombinedRegressor(
            model_dir=model_dir,
            config=config,
            # wide settings
            linear_feature_columns=wide_columns,
            linear_optimizer=linear_optimizer,
            # deep settings
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=dnn_hidden_units,
            dnn_optimizer=dnn_optimizer,
            dnn_dropout=dnn_dropout,
            batch_norm=dnn_batch_norm,
        )
    else:
        raise ValueError(
            "To generate wide model, set wide_columns.\n"
            "To generate deep model, set deep_columns.\n"
            "To generate wide_deep model, set both wide_columns and deep_columns."
        )

    return model

In [87]:
OPTIMIZERS = dict(
    adadelta=tf.keras.optimizers.Adadelta,
    adagrad=tf.keras.optimizers.Adagrad,
    adam=tf.keras.optimizers.Adam,
    ftrl=tf.keras.optimizers.Ftrl,
    rmsprop=tf.keras.optimizers.RMSprop,
    sgd=tf.keras.optimizers.SGD
)

In [88]:
def build_optimizer(name, lr=0.001, **kwargs):
    """Get an optimizer for TensorFlow high-level API Estimator.

    Args:
        name (str): Optimizer name. Note, to use 'Momentum', should specify
        lr (float): Learning rate
        kwargs: Optimizer arguments as key-value pairs

    Returns:
        tf.train.Optimizer
    """
    name = name.lower()

    try:
        optimizer_class = OPTIMIZERS[name]
    except KeyError:
        raise KeyError("Optimizer name should be one of: {}".format(list(OPTIMIZERS)))

    # Set parameters
    params = {}
    if name == "ftrl":
        params["l1_regularization_strength"] = kwargs.get(
            "l1_regularization_strength", 0.0
        )
        params["l2_regularization_strength"] = kwargs.get(
            "l2_regularization_strength", 0.0
        )
    elif name == "rmsprop":
        params["momentum"] = kwargs.get("momentum", 0.0)

    return optimizer_class(learning_rate=lr, **params)

# Build model

In [64]:
# Define wide (linear) and deep (dnn) features
wide_columns, deep_columns = build_feature_columns(
    users=users["UserID"].values,
    items=items["MovieID"].values,
    user_col="UserID",
    item_col="MovieID",
    item_feat_col="Genres",
    crossed_feat_dim=1000,
    user_dim=32,
    item_dim=16,
    item_feat_shape=item_feat_shape,
    model_type="wide_deep"
)

In [65]:
print("Wide feature specs:")
for c in wide_columns:
    print("\t", str(c)[:100], "...")
print("Deep feature specs:")
for c in deep_columns:
    print("\t", str(c)[:100], "...")

Wide feature specs:
	 VocabularyListCategoricalColumn(key='UserID', vocabulary_list=(1, 2, 12, 15, 17, 18, 19, 24, 28, 33, ...
	 VocabularyListCategoricalColumn(key='MovieID', vocabulary_list=(1193, 661, 914, 3408, 2355, 1197, 12 ...
	 CrossedColumn(keys=(VocabularyListCategoricalColumn(key='UserID', vocabulary_list=(1, 2, 12, 15, 17, ...
Deep feature specs:
	 EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='UserID', vocabulary_list=(1, ...
	 EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='MovieID', vocabulary_list=(1 ...
	 NumericColumn(key='Genres', shape=(18,), default_value=None, dtype=tf.float32, normalizer_fn=None) ...


In [79]:
TMP_DIR = TemporaryDirectory()
model_dir = TMP_DIR.name
model_dir

'/var/folders/45/q__m21f163d57l7zhtj1yvvc0000gn/T/tmpkk9yzulm'

In [89]:
model = build_model(
    model_dir=model_dir,
    wide_columns=wide_columns,
    deep_columns=deep_columns,
    linear_optimizer=build_optimizer('adagrad', 0.0621, **{
        'l1_regularization_strength': 0,
        'l2_regularization_strength': 0,
        'momentum': 0,
    }),
    dnn_optimizer=build_optimizer('adadelta', 0.1, **{
        'l1_regularization_strength': 0,
        'l2_regularization_strength': 0,
        'momentum': 0,  
    }),
    dnn_hidden_units=[h for h in [0, 64, 128, 512] if h > 0],
    dnn_dropout=0.8,
    dnn_batch_norm=True,
    log_every_n_iter=max(1, 50000//10),  # log 10 times
    save_checkpoints_steps=max(1, 50000 // 5)
)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/45/q__m21f163d57l7zhtj1yvvc0000gn/T/tmpkk9yzulm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 5000, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train and evaluate model

In [67]:
def filter_by(df, filter_by_df, filter_by_cols):
    """From the input DataFrame (df), remove the records whose target column (filter_by_cols) values are
    exist in the filter-by DataFrame (filter_by_df)

    Args:
        df (pd.DataFrame): Source dataframe.
        filter_by_df (pd.DataFrame): Filter dataframe.
        filter_by_cols (iterable of str): Filter columns.

    Returns:
        pd.DataFrame: Dataframe filtered by filter_by_df on filter_by_cols
    """

    return df.loc[
        ~df.set_index(filter_by_cols).index.isin(
            filter_by_df.set_index(filter_by_cols).index
        )
    ]


def user_item_pairs(
    user_df,
    item_df,
    user_col,
    item_col,
    user_item_filter_df=None,
    shuffle=True,
    seed=None,
):
    """Get all pairs of users and items data.

    Args:
        user_df (pd.DataFrame): User data containing unique user ids and maybe their features.
        item_df (pd.DataFrame): Item data containing unique item ids and maybe their features.
        user_col (str): User id column name.
        item_col (str): Item id column name.
        user_item_filter_df (pd.DataFrame): User-item pairs to be used as a filter.
        shuffle (bool): If True, shuffles the result.
        seed (int): Random seed for shuffle

    Returns:
        pd.DataFrame: All pairs of user-item from user_df and item_df, excepting the pairs in user_item_filter_df
    """

    # Get all user-item pairs
    user_df["key"] = 1
    item_df["key"] = 1
    users_items = user_df.merge(item_df, on="key")

    user_df.drop("key", axis=1, inplace=True)
    item_df.drop("key", axis=1, inplace=True)
    users_items.drop("key", axis=1, inplace=True)

    # Filter
    if user_item_filter_df is not None:
        users_items = filter_by(users_items, user_item_filter_df, [user_col, item_col])

    if shuffle:
        users_items = users_items.sample(frac=1, random_state=seed).reset_index(
            drop=True
        )

    return users_items

In [68]:
ranking_pool = user_item_pairs(
    user_df=users,
    item_df=items,
    user_col="UserID",
    item_col="MovieID",
    user_item_filter_df=train,  # Remove seen items
    shuffle=True
)

In [69]:
ranking_pool.shape

(21634084, 3)

In [70]:
def _dataset(x, y=None, batch_size=128, num_epochs=1, shuffle=False, seed=None):
    if y is None:
        dataset = tf.data.Dataset.from_tensor_slices(x)
    else:
        dataset = tf.data.Dataset.from_tensor_slices((x, y))

    if shuffle:
        dataset = dataset.shuffle(
            1000, seed=seed, reshuffle_each_iteration=True  # buffer size = 1000
        )
    elif seed is not None:
        import warnings

        warnings.warn("Seed was set but `shuffle=False`. Seed will be ignored.")

    return dataset.repeat(num_epochs).batch(batch_size)


def pandas_input_fn(
    df, y_col=None, batch_size=128, num_epochs=1, shuffle=False, seed=None
):
    """Pandas input function for TensorFlow high-level API Estimator.
    This function returns a `tf.data.Dataset` function.

    .. note::
    
        `tf.estimator.inputs.pandas_input_fn` cannot handle array/list column properly.
        For more information, see https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn

    Args:
        df (pd.DataFrame): Data containing features.
        y_col (str): Label column name if df has it.
        batch_size (int): Batch size for the input function.
        num_epochs (int): Number of epochs to iterate over data. If None will run forever.
        shuffle (bool): If True, shuffles the data queue.
        seed (int): Random seed for shuffle.

    Returns:
        tf.data.Dataset function
    """

    X_df = df.copy()
    if y_col is not None:
        y = X_df.pop(y_col).values
    else:
        y = None

    X = {}
    for col in X_df.columns:
        values = X_df[col].values
        if isinstance(values[0], (list, np.ndarray)):
            values = np.array([l for l in values], dtype=np.float32)
        X[col] = values

    return lambda: _dataset(
        x=X,
        y=y,
        batch_size=batch_size,
        num_epochs=num_epochs,
        shuffle=shuffle,
        seed=seed
    )

In [72]:
train_fn = pandas_input_fn(
    df=train,
    y_col="Rating",
    batch_size=32,
    num_epochs=None,  # We use steps=TRAIN_STEPS instead.
    shuffle=True
)

In [90]:
print("Training steps = {}, Batch size = {} (num epochs = {})".format(50000, 32, 5000 * 32 // len(train)))

try:
    model.train(input_fn=train_fn, steps=50000)
except Exception as e:
    print(e)

Training steps = 50000, Batch size = 32 (num epochs = 0)
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/45/q__m21f163d57l7zhtj1yvvc0000gn/T/tmpkk9yzulm/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 15.009094, step = 0
INFO:tensorflow:global_step/sec: 517.555
INFO:tensorflow:loss = 0.85233754, step = 5000 (9.661 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving ch

# Evaluate model

## Item rating prediction

In [95]:
predictions = list(model.predict(input_fn=pandas_input_fn(df=test)))
prediction_df = test.drop("Rating", axis=1)
prediction_df["Rating"] = [p['predictions'][0] for p in predictions]

In [100]:
mean_squared_error(test["Rating"], prediction_df["Rating"])

0.8520599047418799

In [102]:
mean_absolute_error(test["Rating"], prediction_df["Rating"])

0.7395462591053145

## Recommend k items

In [104]:
predictions = list(model.predict(input_fn=pandas_input_fn(df=ranking_pool)))
prediction_df = ranking_pool.copy()
prediction_df["Rating"] = [p['predictions'][0] for p in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/45/q__m21f163d57l7zhtj1yvvc0000gn/T/tmpkk9yzulm/model.ckpt-50000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [105]:
prediction_df["Rating"]

0           3.125630
1           3.579156
2           3.032273
3           2.844368
4           3.249866
              ...   
21634079    3.434063
21634080    3.047624
21634081    3.225728
21634082    2.947160
21634083    3.550170
Name: Rating, Length: 21634084, dtype: float64

In [106]:
def get_top_k_items(
    dataframe, col_user, col_rating, k
):
    """Get the input customer-item-rating tuple in the format of Pandas
    DataFrame, output a Pandas DataFrame in the dense format of top k items
    for each user.
    
    Note:
        If it is implicit rating, just append a column of constants to be
        ratings.

    Args:
        dataframe (pandas.DataFrame): DataFrame of rating data (in the format
        customerID-itemID-rating)
        col_user (str): column name for user
        col_rating (str): column name for rating
        k (int or None): number of items for each user; None means that the input has already been
        filtered out top k items and sorted by ratings and there is no need to do that again.

    Returns:
        pd.DataFrame: DataFrame of top k items for each user, sorted by `col_user` and `rank`
    """
    # Sort dataframe by col_user and (top k) col_rating
    if k is None:
        top_k_items = dataframe
    else:
        top_k_items = (
            dataframe.groupby(col_user, as_index=False)
            .apply(lambda x: x.nlargest(k, col_rating))
            .reset_index(drop=True)
        )
    # Add ranks
    top_k_items["rank"] = top_k_items.groupby(col_user, sort=False).cumcount() + 1
    return top_k_items


def merge_ranking_true_pred(
    rating_true,
    rating_pred,
    col_user,
    col_item,
    col_rating,
    col_prediction,
    relevancy_method,
    k,
    threshold,
):
    """Filter truth and prediction data frames on common users

    Args:
        rating_true (pd.DataFrame): True DataFrame
        rating_pred (pd.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the 
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user (optional)
        threshold (float): threshold of top items per user (optional)

    Returns:
        pd.DataFrame, pd.DataFrame, int: DataFrame of recommendation hits, sorted by `col_user` and `rank`
        DataFrmae of hit counts vs actual relevant items per user number of unique user ids
    """

    # Make sure the prediction and true data frames have the same set of users
    common_users = set(rating_true[col_user]).intersection(set(rating_pred[col_user]))
    rating_true_common = rating_true[rating_true[col_user].isin(common_users)]
    rating_pred_common = rating_pred[rating_pred[col_user].isin(common_users)]
    n_users = len(common_users)

    # Return hit items in prediction data frame with ranking information. This is used for calculating NDCG and MAP.
    # Use first to generate unique ranking values for each item. This is to align with the implementation in
    # Spark evaluation metrics, where index of each recommended items (the indices are unique to items) is used
    # to calculate penalized precision of the ordered items.
    if relevancy_method == "top_k":
        top_k = k
    elif relevancy_method == "by_threshold":
        top_k = threshold
    elif relevancy_method is None:
        top_k = None
    else:
        raise NotImplementedError("Invalid relevancy_method")
    df_hit = get_top_k_items(
        dataframe=rating_pred_common,
        col_user=col_user,
        col_rating=col_prediction,
        k=top_k,
    )
    df_hit = pd.merge(df_hit, rating_true_common, on=[col_user, col_item])[
        [col_user, col_item, "rank"]
    ]

    # count the number of hits vs actual relevant items per user
    df_hit_count = pd.merge(
        df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
        rating_true_common.groupby(col_user, as_index=False)[col_user].agg(
            {"actual": "count"}
        ),
        on=col_user,
    )

    return df_hit, df_hit_count, n_users

In [108]:
df_hit, df_hit_count, n_users = merge_ranking_true_pred(
    rating_true=test,
    rating_pred=prediction_df,
    col_user="UserID",
    col_item="MovieID",
    col_rating="Rating",
    col_prediction="Rating",
    relevancy_method="top_k",
    k=10,
    threshold=None
)

In [116]:
# Normalized Discounted Cumulative Gain (nDCG).

# calculate discounted gain for hit items
df_dcg = df_hit.copy()
# relevance in this case is always 1
df_dcg["dcg"] = 1 / np.log1p(df_dcg["rank"])
# sum up discount gained to get discount cumulative gain
df_dcg = df_dcg.groupby("UserID", as_index=False, sort=False).agg({"dcg": "sum"})
# calculate ideal discounted cumulative gain
df_ndcg = pd.merge(df_dcg, df_hit_count, on=["UserID"])
df_ndcg["idcg"] = df_ndcg["actual"].apply(
    lambda x: sum(1 / np.log1p(range(1, min(x, 10) + 1)))
)

# DCG over IDCG is the normalized DCG
(df_ndcg["dcg"] / df_ndcg["idcg"]).sum() / n_users

0.10541205689683304

In [112]:
# precision at k
(df_hit_count["hit"] / 10).sum() / n_users

0.10006622516556293